# Interacting with Recommenders, Campaigns and Filters <a class="anchor" id="top"></a>

Now that Unicorn Post has trained models for 2 different use cases (User Personalization and Personalized Reranking), we need to integrate them into our application. Amazon Personalize can make recommendations available via an Application Programming Interface (API). In addition, Amazon Personalize includes features that allow you to easily integrate into applications and provide benefits like real time vending of recommendations based on recent application activity.

In this notebook, you will interact with campaigns and filters you created earlier in Amazon Personalize.

1. [Introduction](#intro)
1. [Interact with Recommenders](#interact-recommenders)
1. [Interact with Campaigns](#interact-campaigns)
1. [Filters](#filters)
1. [Create Filters](#create-filters)
1. [Using Filters](#using-filters)
1. [Real-time Events](#real-time)
1. [Wrap Up](#wrapup)

To run this notebook, you need to have run the previous notebooks, [`01_Data_Layer.ipynb`](01_Data_Layer.ipynb), and [`02_Training_Layer.ipynb`](02_Training_Layer.ipynb), where you created a dataset and imported interaction, item, and user metadata data into Amazon Personalize, created recommenders, solutions, and campaigns.

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have two deployed Campaign. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

In this Notebook we will interact with the Campaigns and get recommendations. 

We will interact with filters and send live data to Amazon Personalize to see the effect of real-time interactions on recommendations.

The following diagram shows the resources that we will create in this section. The part we are building in this notebook highlighted in blue with a dashed outline.

![Workflow](Images/03_Inference_Layer_Resources.jpg)

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random
import boto3
import pandas as pd

In [2]:
#retrieves previously stored variables 
%store -r 

In [28]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with Campaigns <a class="anchor" id="interact-recommenders"></a>
[Back to top](#top)

Now that the models have been trained, lets have a look at the recommendations we can get for our users!

### User Personalization Model

"User Personalization " requires a user as input, and it will return the items it thinks the customer is most likely to interact with next.

The cells below will handle getting recommendations from the "User Personalization Model" and rendering the results. Let's see what the recommendations are for a user.

We will be using the `campaignArn`, the `userId`, as well as the number or results we want, `numResults`.

### Select a User

We'll just pick a random user for simplicity. Feel free to change the `user_id` below and execute the following cells with a different user to get a sense for how the recommendations change.

#### Sample User ID's
 -8845298781299428018
 -1032019229384696495
 -1130272294246983140
 344280948527967603
 -445337111692715325

In [4]:
sample_user = str(-8845298781299428018)

In [5]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 5
)

In [6]:
print(get_recommendations_response)

{'ResponseMetadata': {'RequestId': 'e76ff07f-7b7c-47c2-a326-86110e1cf1ea', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 28 Mar 2024 23:28:12 GMT', 'content-type': 'application/json', 'content-length': '613', 'connection': 'keep-alive', 'x-amzn-requestid': 'e76ff07f-7b7c-47c2-a326-86110e1cf1ea'}, 'RetryAttempts': 0}, 'itemList': [{'itemId': '3566197569262766169', 'score': 0.9387915}, {'itemId': '3618205920179577598', 'score': 0.0025508}, {'itemId': '9056727675613132316', 'score': 0.0017862}, {'itemId': '-8502883448980287228', 'score': 0.0014528}, {'itemId': '-1419188045265393093', 'score': 0.0014075}], 'recommendationId': 'RID-cd-44ab-abab-458deb7804a1-CID-482b34'}


A little hard to read - lets make a dataframe

In [7]:
recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])

In [8]:
recommendations_df

,itemId,score
0,3566197569262766169,0.938792
1,3618205920179577598,0.002551
2,9056727675613132316,0.001786
3,-8502883448980287228,0.001453
4,-1419188045265393093,0.001407


Lets lookup what sort of articles are being recommended to this user

In [9]:
articles_mlfeatures['item_id'] = articles_mlfeatures['item_id'].astype('str')

In [10]:
recommendations_df = recommendations_df.merge(articles_mlfeatures, how='left', left_on='itemId', right_on='item_id')

In [11]:
recommendations_df

,itemId,score,creation_timestamp,item_id,lang,article_genre,training_text
0,3566197569262766169,0.938792,1486520793,3566197569262766169,en,cloud provider news,Google extends Gmail API for more granular ema...
1,3618205920179577598,0.002551,1487263968,3618205920179577598,pt,crypto currency,Celcoin allows users to exchange bitcoin for c...
2,9056727675613132316,0.001786,1471905077,9056727675613132316,en,cloud provider news,Amazon CloudFront speeds up content delivery t...
3,-8502883448980287228,0.001453,1486128294,-8502883448980287228,en,cloud provider news,Organization resource enables centralized mana...
4,-1419188045265393093,0.001407,1487069282,-1419188045265393093,en,cloud provider news,Google Cloud Endpoints provides managed API ga...


In [12]:
for i, row in recommendations_df.iterrows():
    print(row['training_text'])
    print("===================================")

Google extends Gmail API for more granular email settings management Google updated the Gmail API with new endpoints to manage filters, aliases, forwarding, signatures, vacation responders, and other granular email settings. This replaces the deprecated Email Settings API. Google has extended the Gmail API with new endpoints for managing email settings like filters, forwarding addresses, IMAP/POP settings, sendas aliases, signatures, and vacation responders. Developers can now retrieve and update signatures for sendas aliases, configure forwarding to external addresses, configure sendas aliases through external providers, use HTML in vacation messages, and manipulate settings for gmail.com accounts. More settings features like mailbox delegate support will be added over time. Most settings endpoints work for any Google Apps or Gmail account, but sensitive operations like modifying aliases or forwarding are restricted to service accounts with domainwide authority. The existing Email Set

What has this user viewed previously

In [13]:
viewed_interactions = interaction_data[interaction_data['user_id'].astype(str) == sample_user].sort_values('timestamp', ascending=False)

In [14]:
viewed_interactions

,timestamp,event_type,item_id,user_id,session_id,user_device_type
69083,1487360168,FOLLOW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69085,1487360167,COMMENT CREATED,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69082,1487359849,VIEW,3566197569262766169,-8845298781299428018,808768479044973017,NonMobile
69722,1487069095,VIEW,-8900113512825364282,-8845298781299428018,8663979798581613597,NonMobile
68453,1485973216,VIEW,-532999578436827210,-8845298781299428018,-5430065457414428568,NonMobile
...,...,...,...,...,...,...
12169,1459345020,VIEW,7973573994178035769,-8845298781299428018,3760091107461406486,NonMobile
18783,1459285852,VIEW,6152652267138213180,-8845298781299428018,-6283148774987755959,NonMobile
692,1459274282,VIEW,-1672166631728511207,-8845298781299428018,-6283148774987755959,NonMobile
684,1459274266,VIEW,-1672166631728511207,-8845298781299428018,-6283148774987755959,NonMobile


lets take the most recent 5 articles interacted with by this user

In [15]:
most_recent_five_articles = viewed_interactions.item_id.unique()[:5].astype(str).tolist()

In [16]:
most_recent_five_articles_metadata = articles_mlfeatures[articles_mlfeatures['item_id'].isin(most_recent_five_articles)]

In [17]:
most_recent_five_articles_metadata

,creation_timestamp,item_id,lang,article_genre,training_text
2869,1481676932,8526042588044002101,en,tech,"Cloud Native enables efficient, automated infr..."
2982,1484838924,7419040071212162906,en,tech,"Google search traffic hack by targeting \""best..."
3040,1485899108,-532999578436827210,en,cloud provider news,IBM launches cloud graph database service usin...
3065,1486520793,3566197569262766169,en,cloud provider news,Google extends Gmail API for more granular ema...
3084,1486999803,-8900113512825364282,en,non tech,Report finds banks lag in customer experience ...


We see that the user has previously red information on cloud applications and google in particular. In fact one of our recommendations is actually in the users recent interaction history. This is not ideal lets use the fitler we created earlier to exclude this data. Also lets make use of the return metadata function to retrieve the information on this article so that the recommendations are easier for us to understand. 

In [45]:
response = personalize.update_campaign(
    campaignArn = workshop_userpersonalization_campaign_arn,
    campaignConfig={
        'enableMetadataWithRecommendations': True
    }
)
print (response)

response = personalize.update_campaign(
    campaignArn = workshop_rerank_campaign_arn,
    campaignConfig={
        'enableMetadataWithRecommendations': True
    }
)
print (response)

{'campaignArn': 'arn:aws:personalize:us-east-1:381491864570:campaign/immersion_day_user_personalization_news_campaign', 'ResponseMetadata': {'RequestId': 'b967d27b-1b69-4102-9ed2-4ab49e74a6d7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 30 Mar 2024 01:03:01 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '118', 'connection': 'keep-alive', 'x-amzn-requestid': 'b967d27b-1b69-4102-9ed2-4ab49e74a6d7'}, 'RetryAttempts': 0}}
{'campaignArn': 'arn:aws:personalize:us-east-1:381491864570:campaign/immersion_day_personalized_ranking_news_campaign', 'ResponseMetadata': {'RequestId': '7d2eb2c1-4331-4295-9ad8-d3af484244b1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 30 Mar 2024 01:03:01 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '118', 'connection': 'keep-alive', 'x-amzn-requestid': '7d2eb2c1-4331-4295-9ad8-d3af484244b1'}, 'RetryAttempts': 0}}


Note you can enable metadata with recommendations when you deploy the campaign for the first time as follows:

```python
user_personalization_create_campaign_response = personalize.create_campaign(
    name = workshop_userpersonalization_campaign_name,
    solutionVersionArn = workshop_userpersonalization_solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {"enableMetadataWithRecommendations": True}    
)
```

In [50]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = workshop_userpersonalization_campaign_arn,
    userId = sample_user,
    numResults = 10,
    metadataColumns = {"ITEMS": ["ARTICLE_GENRE"]}
)

In [51]:
get_recommendations_response

{'ResponseMetadata': {'RequestId': '993c5ae4-a5f9-4c2c-a258-1b0652af47c9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 30 Mar 2024 01:09:07 GMT',
   'content-type': 'application/json',
   'content-length': '1146',
   'connection': 'keep-alive',
   'x-amzn-requestid': '993c5ae4-a5f9-4c2c-a258-1b0652af47c9'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': '3566197569262766169', 'score': 0.9387915},
  {'itemId': '3618205920179577598', 'score': 0.0025508},
  {'itemId': '9056727675613132316', 'score': 0.0017862},
  {'itemId': '-8502883448980287228', 'score': 0.0014528},
  {'itemId': '-1419188045265393093', 'score': 0.0014075},
  {'itemId': '2573252627510191315', 'score': 0.001402},
  {'itemId': '1964631817676172382', 'score': 0.0013023},
  {'itemId': '-5160830061297425098', 'score': 0.0010796},
  {'itemId': '-3870325027932266414', 'score': 0.001076},
  {'itemId': '672199059798181601', 'score': 0.0009841}],
 'recommendationId': 'RID-a4-481c-bf35-875fb75d1df2-CID-482b34'}

To make processing this easier lets make a helper function which retrieves the interaction history for our users.

In [21]:
def return_recs_and_history(user_id, campaign_arn, article_metadata, interactions_data, filter_arn, filter_values):
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = user_id,
        numResults = 5,
        filterArn = filter_arn, #"arn:aws:personalize:us-east-1:256589169164:filter/breakingnews-genre-filter"
        filterValues = filter_values #{"GENRELIST": "\"tech\", \"cloud provider news\""}
    )
    
    recommendations_df = pd.DataFrame.from_records(get_recommendations_response['itemList'])
    
    recommendations_df = recommendations_df.merge(article_metadata, how='left', left_on='itemId', right_on='item_id')
    
    viewed_interactions = interactions_data[interactions_data['user_id'].astype(str) == user_id].sort_values('timestamp', ascending=False)
    
    most_recent_five_articles = viewed_interactions.item_id.unique()[:5].astype(str).tolist()
    
    most_recent_five_articles_metadata = article_metadata[article_metadata['item_id'].isin(most_recent_five_articles)]
    
    return recommendations_df, most_recent_five_articles_metadata

In [24]:
recommendations_df, most_recent_five_articles_metadata = return_recs_and_history(user_id = sample_user,
                                                                                campaign_arn = workshop_userpersonalization_campaign_arn,
                                                                                article_metadata = articles_mlfeatures,
                                                                                interactions_data = interaction_data,
                                                                                filter_arn = "arn:aws:personalize:us-east-1:256589169164:filter/breakingnews-genre-filter",
                                                                                filter_values = {"GENRELIST": "\"tech\", \"cloud provider news\""})